In [33]:
import os
from dotenv import load_dotenv

In [34]:
load_dotenv()
# Set the GROQ API key from the environment variable
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [35]:
from langchain.document_loaders import PyPDFLoader

In [36]:
#load the PDF file
loader = PyPDFLoader("../data/SDG.pdf")
data = loader.load()

In [37]:
len(data)

24

In [38]:
#question generation string
question_gen = ""
for page in data:
    question_gen += page.page_content

In [39]:
question_gen

'IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 yea

In [40]:
#model configuration
from langchain_groq import ChatGroq
model = ChatGroq(
api_key=GROQ_API_KEY,
model="llama3-70b-8192",
)

In [41]:
from langchain.text_splitter import TokenTextSplitter

splitted_questions_gen=TokenTextSplitter(
    chunk_size=10000,
    chunk_overlap=200,
)
chunk_ques_gen= splitted_questions_gen.split_text(question_gen)

In [42]:
chunk_ques_gen

['IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 ye

In [43]:
len(chunk_ques_gen)

1

In [44]:
from langchain.docstore.document import Document
document_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]

In [45]:
document_ques_gen


[Document(metadata={}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence tha

In [46]:
# Create a text splitter for the answer generation
splitter_ans_gen = TokenTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [47]:
len(document_answer_gen)

5

In [48]:
document_answer_gen

[Document(metadata={}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence tha

In [49]:
prompt_template="""
  You are a question generation assistant for coding education and technical exam preparation.
Your task is to carefully read through the following technical material and generate a comprehensive list of exam-style questions.
------------
{text}
------------
🔹 Generate questions that:
Cover all key concepts, logic, and syntax

Include theoretical, practical, and application-based questions

Represent realistic questions a programmer might face in interviews, coding tests, or technical exams

🔹 Include a variety of question types:

❓ Conceptual (e.g., What is...? Why is...?)

🧠 Analytical (e.g., Explain how... Describe the behavior of...)

🧮 Predictive (e.g., What is the output of...? What will happen if...?)

🧑‍💻 Code Completion (e.g., Fill in the missing code...)

✅ Multiple Choice (if relevant)

🔹 Do not include answers, explanations, or summaries.
🔹 Do not leave out any important part of the original content.

QUESTIONS:
 1.
 2.
 3.
 4.

"""

In [50]:
from langchain.prompts import PromptTemplate
PROMPT_QUESTIONS = PromptTemplate(
    template=prompt_template,
    input_variables=["text"]
)

In [51]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [52]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [53]:
from langchain.chains.summarize import load_summarize_chain

ques_gen_chain = load_summarize_chain(llm = model, 
                                          chain_type = "refine", 
                                          verbose = True, 
                                          question_prompt=PROMPT_QUESTIONS, 
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)

In [54]:
question=ques_gen_chain.run(document_answer_gen)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a question generation assistant for coding education and technical exam preparation.
Your task is to carefully read through the following technical material and generate a comprehensive list of exam-style questions.
------------
IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD 
CAME TOGETHER TO FACE THE FUTURE.
And what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. 
Not just in some faraway place, but in their own cities and towns and villages.
They knew things didn’t have to be this way. They knew we had enough 
food to feed the world, but that it wasn’t getting shared. They knew there 
were medicines for HIV and other diseases, but they cost a lot. They knew 
that earthquakes and floods were inevitable, but that the high death 
tolls were not. 
They also knew that billions of people worldwide shared their hope for a 
better future.
So leade

In [55]:
question

"Based on the new context, I've refined the original questions to make them more specific and relevant to the topic of Sustainable Development Goals and global partnership. Here are the refined questions:\n\n1. What is the significance of the global partnership for sustainable development, and how can countries work together to achieve the Sustainable Development Goals by 2030?\n\n2. How can the Sustainable Development Goals promote inclusive and sustainable economic growth, and what are the benefits of achieving this goal?\n\n3. What is the role of global institutions in achieving the Sustainable Development Goals, and how can they facilitate international cooperation and partnerships?\n\n4. How can fundraising and donations contribute to reducing and eradicating poverty, and what are some effective ways to raise money and create awareness about the Sustainable Development Goals?\n\n5. What is the importance of spreading awareness and promoting the Sustainable Development Goals throug

In [56]:
#hugging face embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [57]:
# Create a vector store from the documents
from langchain.vectorstores import FAISS
vector_store = FAISS.from_documents(
    document_answer_gen, 
    embeddings
)

In [58]:
llm_answer_gen= ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama3-70b-8192",
)

In [61]:
ques_list= question.split("\n")
ques_list = [q for q in ques_list if q.strip() != ""]
ques_list

["Based on the new context, I've refined the original questions to make them more specific and relevant to the topic of Sustainable Development Goals and global partnership. Here are the refined questions:",
 '1. What is the significance of the global partnership for sustainable development, and how can countries work together to achieve the Sustainable Development Goals by 2030?',
 '2. How can the Sustainable Development Goals promote inclusive and sustainable economic growth, and what are the benefits of achieving this goal?',
 '3. What is the role of global institutions in achieving the Sustainable Development Goals, and how can they facilitate international cooperation and partnerships?',
 '4. How can fundraising and donations contribute to reducing and eradicating poverty, and what are some effective ways to raise money and create awareness about the Sustainable Development Goals?',
 '5. What is the importance of spreading awareness and promoting the Sustainable Development Goals 

In [66]:
#retrival QA
from langchain.chains import RetrievalQA
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm_answer_gen,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
)

In [67]:
#answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = retrieval_qa.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")

Question:  Based on the new context, I've refined the original questions to make them more specific and relevant to the topic of Sustainable Development Goals and global partnership. Here are the refined questions:
Answer:  I'm happy to help answer your refined questions about Sustainable Development Goals and global partnership. Please go ahead and ask them!
--------------------------------------------------\n\n
Question:  1. What is the significance of the global partnership for sustainable development, and how can countries work together to achieve the Sustainable Development Goals by 2030?
Answer:  According to the provided context, the significance of the global partnership for sustainable development lies in its ability to facilitate international cooperation and collective action towards achieving the Sustainable Development Goals (SDGs) by 2030. The SDGs are a set of 17 goals that aim to end poverty, protect the planet, and ensure peace and prosperity for all.

The text suggest